# **DIA1. Introducción a Apache Airflow**
## Ejercicio: Crear tu primer DAG funcional
---

### ⚠️ IMPORTANTE: Compatibilidad con Windows

**Apache Airflow NO es compatible de forma nativa con Windows.** Airflow está diseñado para sistemas POSIX (Linux/macOS).

#### Opciones para usar Airflow en Windows:

1. **Docker (RECOMENDADO)** - La forma más sencilla y confiable
2. **WSL2 (Windows Subsystem for Linux)** - Ejecutar Linux dentro de Windows
3. **Máquina Virtual** - Instalar Linux en una VM

Este notebook ha sido actualizado para mostrar ambas opciones.

### 1.- Instalación básica de Airflow

#### Opción A: Usando Docker (RECOMENDADO para Windows)

In [ ]:
# OPCIÓN A: Instalación con Docker
# 
# Paso 1: Instalar Docker Desktop para Windows desde:
# https://www.docker.com/products/docker-desktop
#
# Paso 2: Crear un directorio para Airflow
# mkdir airflow-docker
# cd airflow-docker
#
# Paso 3: Descargar el archivo docker-compose.yaml oficial
# curl -LfO 'https://airflow.apache.org/docs/apache-airflow/stable/docker-compose.yaml'
#
# Paso 4: Crear directorios necesarios
# mkdir -p ./dags ./logs ./plugins ./config
#
# Paso 5: Configurar el usuario (en PowerShell)
# echo "AIRFLOW_UID=50000" > .env
#
# Paso 6: Inicializar la base de datos
# docker-compose up airflow-init
#
# Paso 7: Iniciar todos los servicios
# docker-compose up
#
# Paso 8: Acceder a la interfaz web
# http://localhost:8080
# Usuario: airflow
# Contraseña: airflow

print("Para usar Airflow en Windows, sigue los pasos comentados arriba usando Docker.")
print("Docker es la forma más confiable de ejecutar Airflow en Windows.")

#### Opción B: Usando WSL2 (Windows Subsystem for Linux)

In [ ]:
# OPCIÓN B: Instalación en WSL2
#
# Paso 1: Instalar WSL2 (ejecutar en PowerShell como administrador)
# wsl --install
#
# Paso 2: Reiniciar el sistema
#
# Paso 3: Abrir WSL (Ubuntu) y ejecutar los siguientes comandos:
#
# # Actualizar el sistema
# sudo apt update && sudo apt upgrade -y
#
# # Instalar Python y pip
# sudo apt install python3 python3-pip python3-venv -y
#
# # Crear entorno virtual
# python3 -m venv airflow_env
# source airflow_env/bin/activate
#
# # Instalar Airflow
# pip install apache-airflow
#
# # Inicializar base de datos
# airflow db init
#
# # Crear usuario admin
# airflow users create \
#     --username admin \
#     --firstname Admin \
#     --lastname User \
#     --role Admin \
#     --email admin@example.com \
#     --password admin

print("Para usar Airflow con WSL2, sigue los pasos comentados arriba.")
print("WSL2 te permite ejecutar Linux dentro de Windows de forma nativa.")

### 2.- Crear primer DAG

Este código debe guardarse en un archivo `.py` en la carpeta `dags/` de tu instalación de Airflow.

**Ubicación del archivo:**
- Docker: `./dags/mi_primer_dag.py`
- WSL2: `~/airflow/dags/mi_primer_dag.py`

In [ ]:
# Contenido del archivo: dags/mi_primer_dag.py
# NOTA: Este código NO se ejecuta directamente en Jupyter.
# Debe guardarse como archivo .py en la carpeta dags/ de Airflow.

codigo_dag = '''
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta

def saludar():
    """Función que imprime un saludo."""
    print("¡Hola desde Airflow!")
    return "Saludo completado"

# Argumentos por defecto para el DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Definir DAG
with DAG(
    'saludo_diario',
    default_args=default_args,
    description='DAG que saluda cada día',
    schedule_interval=timedelta(days=1),  # Ejecutar diariamente
    start_date=datetime(2024, 1, 1),
    catchup=False,  # No ejecutar ejecuciones pasadas
    tags=['ejemplo', 'saludo'],
) as dag:

    # Tarea 1: Comando bash
    tarea_bash = BashOperator(
        task_id='tarea_bash',
        bash_command='echo "Ejecutando tarea bash a las $(date)"',
    )

    # Tarea 2: Función Python
    tarea_python = PythonOperator(
        task_id='tarea_python',
        python_callable=saludar,
    )

    # Tarea 3: Esperar (simular procesamiento)
    tarea_esperar = BashOperator(
        task_id='tarea_esperar',
        bash_command='sleep 5',
    )

    # Definir orden de ejecución
    tarea_bash >> tarea_python >> tarea_esperar
'''

# Guardar el código en un archivo
print("Código del DAG:")
print(codigo_dag)
print("\n" + "="*60)
print("INSTRUCCIONES:")
print("1. Copia el código de arriba")
print("2. Guárdalo en: dags/mi_primer_dag.py")
print("3. Airflow detectará automáticamente el archivo")
print("="*60)

### 3.- Ejecutar y monitorear

Estos comandos se ejecutan en la terminal, NO en Jupyter.

In [ ]:
# COMANDOS PARA EJECUTAR EN TERMINAL (no en Jupyter)

comandos = '''
# === OPCIÓN A: Si usas Docker ===
# Los servicios ya están corriendo con docker-compose up
# Solo necesitas acceder a http://localhost:8080

# === OPCIÓN B: Si usas WSL2 ===
# Terminal 1: Iniciar scheduler
airflow scheduler

# Terminal 2: Iniciar webserver
airflow webserver --port 8080

# Terminal 3: Activar y ejecutar DAG
airflow dags unpause saludo_diario
airflow dags trigger saludo_diario

# Ver lista de DAGs
airflow dags list

# Ver tareas de un DAG
airflow tasks list saludo_diario

# Ver estado de ejecuciones
airflow dags list-runs -d saludo_diario
'''

print("COMANDOS PARA EJECUTAR EN TERMINAL:")
print("="*60)
print(comandos)
print("="*60)

### 4.- Ver resultados

La interfaz web de Airflow es la forma principal de monitorear tus DAGs.

In [ ]:
# ACCEDER A LA INTERFAZ WEB

instrucciones = '''
1. Abrir navegador web
2. Ir a: http://localhost:8080
3. Iniciar sesión:
   - Usuario: airflow (o admin si usas WSL2)
   - Contraseña: airflow (o admin si usas WSL2)

4. Navegar por la interfaz:
   - DAGs: Lista de todos los DAGs disponibles
   - Graph View: Visualización del flujo de tareas
   - Tree View: Historial de ejecuciones
   - Gantt: Diagrama de Gantt de las tareas
   - Code: Ver el código del DAG
   - Logs: Ver los logs de cada tarea

5. Ejecutar manualmente:
   - Click en el DAG "saludo_diario"
   - Click en el botón "Play" (▶) en la esquina superior derecha
   - Seleccionar "Trigger DAG"

6. Ver logs:
   - Click en una tarea en el Graph View
   - Click en "Log"
   - Ver la salida de la ejecución
'''

print("CÓMO USAR LA INTERFAZ WEB DE AIRFLOW:")
print("="*60)
print(instrucciones)
print("="*60)

### 5.- Conceptos clave de Airflow

Entender estos conceptos es fundamental para trabajar con Airflow.

In [ ]:
conceptos = '''
📚 CONCEPTOS CLAVE:

1. DAG (Directed Acyclic Graph):
   - Grafo dirigido acíclico que define el flujo de trabajo
   - Contiene tareas y sus dependencias
   - No puede tener ciclos (A -> B -> C -> A ❌)

2. Task (Tarea):
   - Unidad básica de trabajo en Airflow
   - Puede ser: BashOperator, PythonOperator, etc.
   - Tiene un estado: success, failed, running, etc.

3. Operator (Operador):
   - Plantilla para crear tareas
   - Tipos comunes:
     * BashOperator: Ejecuta comandos bash
     * PythonOperator: Ejecuta funciones Python
     * EmailOperator: Envía emails
     * SQLOperator: Ejecuta queries SQL

4. Schedule Interval:
   - Frecuencia de ejecución del DAG
   - Puede ser: cron expression o timedelta
   - Ejemplos:
     * '@daily': Una vez al día
     * '@hourly': Una vez por hora
     * '0 0 * * *': Cron expression (medianoche)

5. Execution Date:
   - Fecha lógica de ejecución (no la fecha real)
   - Importante para procesar datos históricos

6. Catchup:
   - Si True: Ejecuta todas las ejecuciones pasadas
   - Si False: Solo ejecuta desde ahora en adelante

7. Dependencies (Dependencias):
   - Definen el orden de ejecución
   - Sintaxis: task1 >> task2 >> task3
   - O: task1.set_downstream(task2)
'''

print(conceptos)

### 6.- Troubleshooting (Solución de problemas)

In [ ]:
troubleshooting = '''
🔧 PROBLEMAS COMUNES Y SOLUCIONES:

1. "ModuleNotFoundError: No module named 'airflow'"
   Solución:
   - Asegúrate de estar en el entorno virtual correcto
   - Reinstala: pip install apache-airflow

2. "AttributeError: module 'os' has no attribute 'register_at_fork'"
   Solución:
   - Estás en Windows. Usa Docker o WSL2
   - Airflow NO funciona nativamente en Windows

3. "DAG no aparece en la interfaz web"
   Solución:
   - Verifica que el archivo esté en la carpeta dags/
   - Revisa los logs: airflow dags list-import-errors
   - Espera ~30 segundos (Airflow escanea cada 30s)

4. "Puerto 8080 ya está en uso"
   Solución:
   - Usa otro puerto: airflow webserver --port 8081
   - O detén el proceso: lsof -ti:8080 | xargs kill -9

5. "Tarea falla sin razón aparente"
   Solución:
   - Revisa los logs en la interfaz web
   - Verifica las dependencias de Python
   - Prueba la función fuera de Airflow primero

6. "Base de datos bloqueada (SQLite)"
   Solución:
   - SQLite no es para producción
   - Usa PostgreSQL o MySQL en producción
   - Para desarrollo: reinicia scheduler y webserver
'''

print(troubleshooting)

### 7.- Próximos pasos

Una vez que tengas tu primer DAG funcionando, puedes explorar:

In [ ]:
proximos_pasos = '''
🚀 PRÓXIMOS PASOS:

1. Explorar más operadores:
   - EmailOperator: Enviar notificaciones
   - PostgresOperator: Ejecutar queries SQL
   - HttpOperator: Hacer peticiones HTTP
   - DockerOperator: Ejecutar contenedores

2. Trabajar con XComs:
   - Compartir datos entre tareas
   - Pasar resultados de una tarea a otra

3. Usar Variables y Connections:
   - Almacenar configuraciones
   - Conectar a bases de datos externas

4. Implementar branching:
   - BranchPythonOperator
   - Ejecutar tareas condicionalmente

5. Configurar alertas:
   - Email en caso de fallo
   - Integración con Slack

6. Optimizar performance:
   - Paralelización de tareas
   - Pools y prioridades

7. Desplegar en producción:
   - Usar PostgreSQL/MySQL
   - Configurar CeleryExecutor
   - Implementar monitoring
'''

print(proximos_pasos)